# Lab 1. Forecasting Agent

## Introduction

In this notebook we show you how to create your first sub-agent on [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/).

Amazon Bedrock Agents enable generative AI applications to execute multi-step business tasks using natural language.

In our first example we will create a forecasting agent, where customers can ask the agent to return information about their current energy consumption and forecast of it. 

The following represents the piece of architecture that will be built on this module.

![Forecast Agent Architecture](img/forecast_agent.png)

In this example, we will also enable our agent to use code intepretation capabilities to do basic calculations based on the energy usage and its forecast data. We are also using [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) to provide documentation about the forecasting model and its capabilities.

For completion reasons, we assume that the energy forecast has already been done outside the scope of this agent using a ML model hosted on SageMaker

## Setup


Before we begin, please enable the following models in Bedrock via this link 
https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/modelaccess

Titan Image Generator G1
Claude 3 Sonnet
Claude 3 Haiku


#### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [ ]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

Check your boto3 version

In [1]:
!pip freeze | grep boto3

boto3 @ file:///home/conda/feedstock_root/build_artifacts/boto3_1739930060723/work


In [7]:
!python3 -m pip install --force-reinstall --no-cache -q -r ../requirements.txt


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.29.1 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.20.0 requires botocore<1.36.24,>=1.36.20, but you have botocore 1.37.12 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.2 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-m

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [15]:
import boto3
import os
import json
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
#    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

curr_month = datetime.now()

In [9]:
forecast_agent_name = f"forecast-{agent_suffix}"

forecast_lambda_name = f"fn-forecast-agent-{agent_suffix}"

forecast_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{forecast_agent_name}'

dynamodb_table = f"{forecast_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "day"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{forecast_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information on how forecasting process is done"
bucket_name = f'forecast-agent-kb-{suffix}'


### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [10]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

absl-py @ file:///home/conda/feedstock_root/build_artifacts/absl-py_1733730548347/work
accelerate @ file:///home/conda/feedstock_root/build_artifacts/accelerate_1725632260220/work
adagio @ file:///home/conda/feedstock_root/build_artifacts/adagio_1734649631689/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1740636318979/work
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1713964843148/work
aiohttp-cors @ file:///home/conda/feedstock_root/build_artifacts/aiohttp-cors_1734421108650/work
aioitertools @ file:///home/conda/feedstock_root/build_artifacts/aioitertools_1735329051909/work
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1734342155601/work
aiosqlite @ file:///home/conda/feedstock_root/build_artifacts/aiosqlite_1682491975081/work
alembic @ file:///home/conda/feedstock_root/build_artifacts/alembic_1741181834993/work
altair @ file:///home/conda/feedstock_root/build_artifacts/altair-split_1734244716962/work

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [11]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB forecast-us-west-2-817-kb
Step 1 - Creating or retrieving forecast-agent-kb-us-west-2-817724494920 S3 bucket for Knowledge Base documents
Creating bucket forecast-agent-kb-us-west-2-817724494920
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_652) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '321',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Fri, 14 Mar 2025 09:44:18 '
                                                 'GMT',
                                         'x-amzn-requestid': 'b4db2bb7-8f9e-4294-b444-22a58545f041'},
                        'HTTPStatusCode': 200,
                        'Req

## Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [12]:
path = "kb_documents"

# Check whether the specified path exists or not
is_exist = os.path.exists(path)
if not is_exist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

The kb_documents directory already exists!


Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [13]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            {
                "text": prompt
            }
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close()

### Generating data prompt
Generating one file with forecasting info using the LLM model

In [16]:
text_generation_energy_instructions = '''
    You will be act as data-scientist that knows how to do machine learning
    forecasting using Python and scikit learn. You will generate a step-by-step
    on how to create a forecast process for a time-series data.

    This data has the following json structure:
    {
        "customer_id": "1",
        "day": "2024/06/01",
        "sumPowerReading": "120.0",
        "kind":"measured"
    }

    Choose one forecast algorithm, that works on scikit-learn, explain the
    details on how to create a step-by-step forecast, with code sample,
    showcasing how to run forecast on this data.

    Include some explanation on how to understand the forecasted values and
    how to decide the factors driving those values.

    Answer only with the step-by-step, avoid answer with afirmations like:
    "OK, I can generate it," or "Yes, please find following example."
    Be direct and only reply the step-by-step.
'''

solar_energy_file_name = 'forecasting-info.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)

print("Generated data to be stored in the KB:\n", response_message['content'][0]['text'])
write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    response_message['content'][0]['text']
)

Generated data to be stored in the KB:
 1. Load the time-series data into a pandas DataFrame:

```python
import pandas as pd

data = [
    {"customer_id": "1", "day": "2024/06/01", "sumPowerReading": 120.0, "kind": "measured"},
    {"customer_id": "1", "day": "2024/06/02", "sumPowerReading": 125.0, "kind": "measured"},
    # ... add more data points
]

df = pd.DataFrame(data)
df['day'] = pd.to_datetime(df['day'])
df.set_index('day', inplace=True)
```

2. Split the data into train and test sets:

```python
from sklearn.model_selection import train_test_split

train_size = int(len(df) * 0.8)
train_data, test_data = train_test_split(df, train_size=train_size, shuffle=False)
```

3. Choose a forecasting algorithm from scikit-learn. For this example, we'll use the ExponentialSmoothing model:

```python
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model = ExponentialSmoothing(train_data['sumPowerReading'], seasonal_periods=7, trend='add', seasonal='add')
```

4. Fit the mode

### Uploading data to s3
Uploading generated files into an Amazon S3 Bucket.

In [17]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [19]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_documents/forecasting-info.txt to forecast-agent-kb-us-west-2-817724494920
{ 'dataSourceId': 'N6HTGRRA8O',
  'ingestionJobId': 'DQZCZD7OQA',
  'knowledgeBaseId': 'VYWWQTEL1I',
  'startedAt': datetime.datetime(2025, 3, 14, 10, 6, 2, 295789, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 3, 14, 10, 6, 2, 295789, tzinfo=tzlocal())}
{ 'dataSourceId': 'N6HTGRRA8O',
  'ingestionJobId': 'DQZCZD7OQA',
  'knowledgeBaseId': 'VYWWQTEL1I',
  'startedAt': datetime.datetime(2025, 3, 14, 10, 6, 2, 295789, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
            

## Creating Agent

Create the forecast agent that will have an `Amazon Bedrock Knowledge Base` with information on how forecast process is done as well as the `action groups` to handle the user requests and `code interpretation` capabilities for calculating basic user's requests such as increase in consumption.

In order to have accurate agents, it is important to set unambiguous instructions of what the agent should do and what it should not do. It is also important to provide clear definitions for when the agent should use the knowledge bases and action groups available to it.

We will provide the following instructions to our agent:
```
You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself.
6. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user
```

We will also connect a knowledge base for the explanation of the forecasting methodologies with the following instructions:
```
Access this knowledge base when needing to explain specific forecast generation methodology.
```

And we will make the following tool available to the agent:
- `get_forecasted_consumption`: Gets the next 3 months energy usage forecast
- `get_historical_consumption`: Gets energy usage history to date
- `get_consumption_statistics`: Gets current month usage analytics
- `update_forecasting`: Updates the energy forecast for a specific month


In [20]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [21]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Access this knowledge base when needing to explain specific forecast generation methodology."""
}

In [22]:
agent_description = """You are a energy usage forecast bot.
You can retrieve historical energy consumption, forecasted consumption, usage statistics and update a forecast for a specific user"""

agent_instruction = """You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself.
6. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

forecast_agent = agents.create_agent(
    forecast_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=True
)

forecast_agent

Waiting for agent status to change. Current status CREATING
Agent id DIUTRVOHRF current status: NOT_PREPARED


('DIUTRVOHRF',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:817724494920:agent-alias/DIUTRVOHRF/TSTALIASID')

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [23]:
agents.associate_kb_with_agent(
    forecast_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

### Creating Lambda

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group. You can find more information on how to use action groups to define actions that your agent can perform [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html)

On this block, we're going to generate Lambda function Code:

In [24]:
%%writefile forecast.py
import boto3
import json
import os

from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')
truncated_month = datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def trunc_datetime(month,year):
    return datetime.today().replace(year =int(year), month=int(month), day=1, hour=0, minute=0, second=0, microsecond=0)

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(
    table_name: str, 
    pk_field: str,
    pk_value: str,
    sk_field: str=None, 
    sk_value: str=None,
    attr_key: str=None,
    attr_val: str=None
):
    try:

        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_field:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).eq(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        if attr_key:
            attr_expression = Attr(attr_key).eq(attr_val)
            query_data = table.query(
                KeyConditionExpression=key_expression,
                FilterExpression=attr_expression
            )
        else:
            query_data = table.query(
                KeyConditionExpression=key_expression
            )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def get_forecasted_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="forecasted")

def get_historical_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="measured")

def get_consumption_statistics(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         dynamodb_sk, 
                         truncated_month.strftime('%Y/%m/%d'))

def update_forecasting(customer_id, month, year, usage):
    current_date = trunc_datetime(month, year)
    if  current_date >= truncated_month:
        item = {
            'customer_id': customer_id,
            'day': current_date.strftime('%Y/%m/%d'),
            'sumPowerReading': Decimal(usage),
            'kind': 'forecasted'
        }
        put_dynamodb(dynamodb_table, item)
        return "Day: {} updated for customer: {}".format(current_date.strftime('%Y/%m/%d'), customer_id)
    else:
        return "You're trying to change a past date: {} for customer: {}, which is not allowed".format(current_date.strftime('%Y/%m/%d'), customer_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'get_forecasted_consumption':
        result = get_forecasted_consumption(customer_id)
    elif function == 'get_historical_consumption':
        result = get_historical_consumption(customer_id)
    elif function == 'get_consumption_statistics':
        result = get_consumption_statistics(customer_id)
    elif function == 'update_forecasting':
        month = get_named_parameter(event, "month")
        year = get_named_parameter(event, "year")
        usage = get_named_parameter(event, "usage")
        result = update_forecasting(customer_id, month, year, usage)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Overwriting forecast.py


### Defining available actions

Next we will define the available actions that an agent can perform using [Function Details](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html). You can also do this task using OpenAPI Schemas, which can be very useful if you already have an OpenAPI schema available for your application.

When creating your function details, it is important to provide clear descriptions for the function and for its parameters, as your agent depends on them to correctly orchestrate the tasks to be executed

In [25]:
functions_def = [
    {
        "name": "get_forecasted_consumption",
        "description": """Gets the next 3 months energy usage forecast""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_historical_consumption",
        "description": """Gets energy usage history to date""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_consumption_statistics",
        "description": """Gets current month usage analytics""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "update_forecasting",
        "description": """Updates the energy forecast for a specific month""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "month": {
                "description": "Target update month. In the format MM",
                "required": True,
                "type": "integer"
            },
            "year": {
                "description": "Target update year. In the format YYYY",
                "required": True,
                "type": "integer"
            },
            "usage": {
                "description": "New consumption value",
                "required": True,
                "type": "integer"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [26]:
agents.add_action_group_with_lambda(
    agent_name=forecast_agent_name,
    lambda_function_name=forecast_lambda_name,
    source_code_file="forecast.py",
    agent_functions=functions_def,
    agent_action_group_name="forecast_consumption_actions",
    agent_action_group_description="Function to get usage forecast for a user ",
    dynamo_args=dynamoDB_args
)

## Loading data to DynamoDB

Now that we've created our agent, let's load some generated data to DynamoDB. That will allow the agent to interact with some live data to perform actions

In [27]:
agents.generate_fake_data_dynamodb()

In [28]:
with open("1_user_sample_data.json") as f:
    table_items = [json.loads(line) for line in f]
    
agents.load_dynamodb(dynamodb_table, table_items)

Testing that data was loaded on DynamoDB

In [29]:
today_query = f"{curr_month.year}/{curr_month.month:02d}/01"

resp = agents.query_dynamodb(
    dynamodb_table, dynamodb_pk, '1', dynamodb_sk, today_query
)
resp

[{'customer_id': '1',
  'day': '2025/03/01',
  'kind': 'measured',
  'sumPowerReading': '106'}]

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing get forecast
First let's test the get forecast action

In [30]:
%%time
response = agents.invoke(
    """can you give me my forecasted energy consumption? 
    How does it compare with my past energy usage? My customer id is 1""", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: dbb084df-d18a-4552-96fc-2436a6b2040c
invokeAgent API session ID: 7acc5df4-00b5-11f0-be7b-62f6d0df43a1
---- Step 1 ----
Took 3.9s, using 2434 tokens (in: 2290, out: 144) to complete prior action, observe, orchestrate.
Okay, let me get your forecasted energy consumption and compare it to your historical usage.
Using tool: get_forecasted_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'day': '2025/04/01', 'kind': 'forecasted', 'sumPowerReading': '122'}, {'customer_id': '1', 'day': '2025/05/01', 'kind': 'forecasted', 'sumPowerReading': '136'}, {'customer_id': '1', 'day': '2025/06/01', 'kind': 'forecasted', 'sumPowerReading': '108'}, {'customer_id': '1', 'day'...

Using tool: get_historical_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'day': '2024/11/01', 'kind': 'measured', 'sumPowerReading': '182'}, {

Generated code                                                                                                     

                                                                                                                   
                                                                                                                   
 import pandas as pd                                                                                               
                                                                                                                   
 # Convert lists to dataframes                                                                                     
 forecasted_df = pd.DataFrame(forecasted_data)                                                                     
 historical_df = pd.DataFrame(historical_data)                                                                     
                                                                                                                   
 # Convert date strings to datetime                                                                                
 forecasted_df['day'] = pd.to_datetime(forecasted_df['day'])                                                       
 historical_df['day'] = pd.to_datetime(historical_df['day'])                                                       
                                                                                                                   
 # Calculate monthly sums                                                                                          
 forecasted_monthly = forecasted_df.groupby(pd.Grouper(key='day', freq='M'))['sumPowerReading'].sum().reset_index( 
 historical_monthly = historical_df.groupby(pd.Grouper(key='day', freq='M'))['sumPowerReading'].sum().reset_index( 
                                                                                                                   
 # Print results                                                                                                   
 print('Forecasted monthly consumption:')                                                                          
 print(forecasted_monthly)                                                                                         
 print('Historical monthly consumption:')                                                                          
 print(historical_monthly)                                                                                         
                                                                                                                   

---- Step 3 ----
Took 18.0s, using 4215 tokens (in: 3518, out: 697) to complete prior action, observe, orchestrate.
Oops, my previous code execution failed because I did not define the forecasted_data and historical_data variables first. Let me try again after defining those from the function results:


Generated code                                                                                                     

                                                                                                                   
 import pandas as pd                                                                                               
                                                                                                                   
 # Define data from function results                                                                               
 forecasted_data = [{'customer_id': '1', 'day': '2025/04/01', 'kind': 'forecasted', 'sumPowerReading': '122'},     
 {'customer_id': '1', 'day': '2025/05/01', 'kind': 'forecasted', 'sumPowerReading': '136'}, {'customer_id': '1',   
 'day': '2025/06/01', 'kind': 'forecasted', 'sumPowerReading': '108'}, {'customer_id': '1', 'day': '2025/07/01',   
 'kind': 'forecasted', 'sumPowerReading': '116'}]                                                                  
                                                                                                                   
 historical_data = [{'customer_id': '1', 'day': '2024/11/01', 'kind': 'measured', 'sumPowerReading': '182'},       
 {'customer_id': '1', 'day': '2024/12/01', 'kind': 'measured', 'sumPowerReading': '116'}, {'customer_id': '1',     
 'day': '2025/01/01', 'kind': 'measured', 'sumPowerReading': '160'}, {'customer_id': '1', 'day': '2025/02/01',     
 'kind': 'measured', 'sumPowerReading': '108'}, {'customer_id': '1', 'day': '[38;2;230;219;116;

---- Step 4 ----
Took 7.1s, using 4720 tokens (in: 4416, out: 304) to complete prior action, observe, orchestrate.
The code execution was successful this time. I now have the forecasted and historical monthly energy consumption summarized. To provide a comparison, I will analyze the values:
Final response:
Based on the data, here is how your forecasted energy consumption for the next few months compares to your historical usage:

For April 2025, your forecasted consumption is 122 units, which is lower than your actual consumption in March 2025 (106 units) and January 2025 (160 units), but higher than ...
Agent made a total of 4 LLM calls, using 14606 tokens (in: 13143, out: 1463), and took 36.5 total seconds
Based on the data, here is how your forecasted energy consumption for the next few months compares to your historical usage:

For April 2025, your forecasted consumption is 122 units, which is lower than your actual consumption in March 2025 (106 units) and January 2025 (160 units)

In [ ]:
time.sleep(60)

### Testing get historical consumption
Now we can test the historical energy consumption and also use code interpretation calculate the average energy spending in summer months

In [31]:
%%time
response = agents.invoke(
    "can you give me my past energy consumption? What is my average spending on summer months? My customer id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 7a6f94b3-f6a3-4e87-9cae-029de8ce36e4
invokeAgent API session ID: 7acc5df4-00b5-11f0-be7b-62f6d0df43a1
---- Step 1 ----
Took 3.6s, using 4847 tokens (in: 4747, out: 100) to complete prior action, observe, orchestrate.
To get your past energy consumption history and calculate the average spending for summer months, I will need to retrieve the historical data and perform some analysis on it.
Using tool: get_historical_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'day': '2024/11/01', 'kind': 'measured', 'sumPowerReading': '182'}, {'customer_id': '1', 'day': '2024/12/01', 'kind': 'measured', 'sumPowerReading': '116'}, {'customer_id': '1', 'day': '2025/01/01', 'kind': 'measured', 'sumPowerReading': '160'}, {'customer_id': '1', 'day': '202...

---- Step 2 ----
Took 13.2s, using 5633 tokens (in: 5103, out: 530) to complete prior action, observe, orchestrate.
The historical data only g

Generated code                                                                                                     

                                                                                                                   
 import pandas as pd                                                                                               
                                                                                                                   
 # Convert list to dataframe                                                                                       
 historical_data = [{'customer_id': '1', 'day': '2024/11/01', 'kind': 'measured', 'sumPowerReading': '182'},       
 {'customer_id': '1', 'day': '2024/12/01', 'kind': 'measured', 'sumPowerReading': '116'}, {'customer_id': '1',     
 'day': '2025/01/01', 'kind': 'measured', 'sumPowerReading': '160'}, {'customer_id': '1', 'day': '2025/02/01',     
 'kind': 'measured', 'sumPowerReading': '108'}, {'customer_id': '1', 'day': '2025/03/01', 'kind': 'measured',      
 'sumPowerReading': '106'}]                                                                                        
 historical_df = pd.DataFrame(historical_data)                                                                     
                                                                                                                   
 # Convert date to datetime                                                                                        
 historical_df['day'] = pd.to_datetime(historical_df['day'])                                                       
                                                                                                                   
 # Print historical consumption                                                                                    
 print('Past Energy Consumption:')                                                                                 
 print(historical_df)                                                                                              
                                                                                                                   
 # Define summer months                                                                                            
 summer_months = [6, 7, 8]                                                                                         
                                                                                                                   
 # Filter for summer and calculate average                                                                         
 if any(historical_df['day'].dt.month.isin(summer_months)):                                                        
     summer_df = historical_df[historical_df['day'].dt.month.

---- Step 3 ----
Took 6.5s, using 5997 tokens (in: 5801, out: 196) to complete prior action, observe, orchestrate.
Final response:
Here is an overview of your past energy consumption history:

November 2024: 182 units
December 2024: 116 units  
January 2025: 160 units
February 2025: 108 units
March 2025: 106 units

The data provided does not include any summer months (June, July, August), so I could not calculate your average s...
Agent made a total of 3 LLM calls, using 16477 tokens (in: 15651, out: 826), and took 23.3 total seconds
Here is an overview of your past energy consumption history:

November 2024: 182 units
December 2024: 116 units  
January 2025: 160 units
February 2025: 108 units
March 2025: 106 units

The data provided does not include any summer months (June, July, August), so I could not calculate your average spending for those months. Once data for the summer period becomes available, I can provide the average consumption during those warmer months.

For now, based o

In [ ]:
time.sleep(60)

In [32]:
forecast_agent[0]

'DIUTRVOHRF'

### Testing knowledge base access
Now let's check the knowledge base access by asking a question about the forecasting algorithm

In [33]:
%%time
response = agents.invoke(
    "What's algorithm used for forecast?", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 2da62d05-b4f7-423d-b2e9-469e61272954
invokeAgent API session ID: 7acc5df4-00b5-11f0-be7b-62f6d0df43a1
---- Step 1 ----
Took 5.1s, using 6137 tokens (in: 6005, out: 132) to complete prior action, observe, orchestrate.
The user is asking about the algorithm or methodology used to generate the energy consumption forecasts. I don't have direct access to that information, but I can search the provided knowledge base to see if there are any relevant details.
---- Step 2 ----
Took 5.6s, using 264 tokens (in: 0, out: 264) to complete prior action, observe, orchestrate.
---- Step 3 ----
Took 5.7s, using 6712 tokens (in: 6499, out: 213) to complete prior action, observe, orchestrate.
Final response:



The algorithm used to forecast future energy consumption is the ExponentialSmoothing model from the statsmodels.tsa.holtwinters library. This is a time series forecasting algorithm that can capture seasonality, trend, and other patterns in the historical data.

The key 

In [ ]:
time.sleep(60)

### Testing forecast update
Now we can test the functionality to update the expected forecasting

In [34]:
%%time

future_2m = curr_month + relativedelta(months=2)
future_2m_formatted = future_2m.strftime("%Y/%m")

response = agents.invoke(
    f"Can you update my forecast for month {future_2m_formatted}? I will be travelling and my estimate will be 50. My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 60162b77-5482-41aa-b5cb-8b26907f556c
invokeAgent API session ID: 7acc5df4-00b5-11f0-be7b-62f6d0df43a1
---- Step 1 ----
Took 4.2s, using 6871 tokens (in: 6744, out: 127) to complete prior action, observe, orchestrate.
The user has requested to update the forecasted energy consumption for May 2025 to 50 units. I can use the provided function to make this update.
Using tool: update_forecasting with these inputs:
[{'name': 'month', 'type': 'integer', 'value': '05'}, {'name': 'year', 'type': 'integer', 'value': '2025'}, {'name': 'usage', 'type': 'integer', 'value': '50'}, {'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
Day: 2025/05/01 updated for customer: 1...

---- Step 2 ----
Took 3.8s, using 7014 tokens (in: 6942, out: 72) to complete prior action, observe, orchestrate.
The forecast for May 2025 has been successfully updated to 50 units based on the function output.
Final response:
Your energy consumption forecast for May 2025 has be

In [ ]:
time.sleep(60)

### Confirming that forecast was updated
After updating our forecast, let's check that the forecast was updated and plot a new graph

In [37]:
%%time
response = agents.invoke(
    "Can you give me my forecasted energy consumption month by month? My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: 1776b95d-4eeb-460f-9ba2-8ce252653120
invokeAgent API session ID: 7acc5df4-00b5-11f0-be7b-62f6d0df43a1
---- Step 1 ----
Took 4.9s, using 7132 tokens (in: 7035, out: 97) to complete prior action, observe, orchestrate.
To get the forecasted energy consumption month by month, I will call the get_forecasted_consumption function again after the update.
Using tool: get_forecasted_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'day': '2025/04/01', 'kind': 'forecasted', 'sumPowerReading': '122'}, {'customer_id': '1', 'day': '2025/05/01', 'kind': 'forecasted', 'sumPowerReading': Decimal('50')}, {'customer_id': '1', 'day': '2025/06/01', 'kind': 'forecasted', 'sumPowerReading': '108'}, {'customer_id': '1...

---- Step 2 ----
Took 10.4s, using 7743 tokens (in: 7355, out: 388) to complete prior action, observe, orchestrate.
The function has returned the updated forecasted consumption values f

Generated code                                                                                                     

                                                                                                                   
 import pandas as pd                                                                                               
 from datetime import datetime                                                                                     
                                                                                                                   
 forecasted_data = [{'customer_id': '1', 'day': '2025/04/01', 'kind': 'forecasted', 'sumPowerReading': '122'},     
 {'customer_id': '1', 'day': '2025/05/01', 'kind': 'forecasted', 'sumPowerReading': Decimal('50')}, {'customer_id' 
 '1', 'day': '2025/06/01', 'kind': 'forecasted', 'sumPowerReading': '108'}, {'customer_id': '1', 'day':            
 '2025/07/01', 'kind': 'forecasted', 'sumPowerReading': '116'}]                                                    
                                                                                                                   
 df = pd.DataFrame(forecasted_data)                                                                                
 df['day'] = pd.to_datetime(df['day'], format='%Y/%m/%d')                                                          
                                                                                                                   
 for idx, row in df.iterrows():                                                                                    
     month = row['day'].month                                                                                      
     year = row['day'].year                                                                                        
     forecast = row['sumPowerReading']                                                                             
     print(f"{datetime(year, month, 1).strftime('%B %Y')}: {forecast} units")                                      
                                                                                                                   

---- Step 3 ----
Took 9.2s, using 8574 tokens (in: 8199, out: 375) to complete prior action, observe, orchestrate.
Oops, my previous code had an error because the Decimal type was not defined. Let me try again after converting the data correctly:


Generated code                                                                                                     

                                                                                                                   
 import pandas as pd                                                                                               
 from datetime import datetime                                                                                     
                                                                                                                   
 forecasted_data = [{'customer_id': '1', 'day': '2025/04/01', 'kind': 'forecasted', 'sumPowerReading': 122},       
 {'customer_id': '1', 'day': '2025/05/01', 'kind': 'forecasted', 'sumPowerReading': 50}, {'customer_id': '1', 'day 
 '2025/06/01', 'kind': 'forecasted', 'sumPowerReading': 108}, {'customer_id': '1', 'day': '2025/07/01', 'kind':    
 'forecasted', 'sumPowerReading': 116}]                                                                            
                                                                                                                   
 df = pd.DataFrame(forecasted_data)                                                                                
 df['day'] = pd.to_datetime(df['day'], format='%Y/%m/%d')                                                          
                                                                                                                   
 for idx, row in df.iterrows():                                                                                    
     month = row['day'].month                                                                                      
     year = row['day'].year                                                                                        
     forecast = row['sumPowerReading']                                                                             
     print(f"{datetime(year, month, 1).strftime('%B %Y')}: {forecast} units")                                      
                                                                                                                   

---- Step 4 ----
Took 6.3s, using 8782 tokens (in: 8682, out: 100) to complete prior action, observe, orchestrate.
Final response:
Here are your forecasted monthly energy consumption values:

April 2025: 122 units
May 2025: 50 units (updated value)
June 2025: 108 units  
July 2025: 116 units

The forecast for May 2025 has been updated to 50 units based on your provided estimate for that month due to travelling. The other months...
Agent made a total of 4 LLM calls, using 32231 tokens (in: 31271, out: 960), and took 31.0 total seconds
Here are your forecasted monthly energy consumption values:

April 2025: 122 units
May 2025: 50 units (updated value)
June 2025: 108 units  
July 2025: 116 units

The forecast for May 2025 has been updated to 50 units based on your provided estimate for that month due to travelling. The other months remain as originally forecasted.
CPU times: user 56.3 ms, sys: 7.87 ms, total: 64.2 ms
Wall time: 31 s


In [ ]:
time.sleep(60)

### Testing forecasting statistics
Finally, let's test the get stats functionality

In [40]:
%%time
response = agents.invoke(
    "can you give me my current consumption? My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

invokeAgent API request ID: be8628a5-5779-418e-8c65-a88402d742a3
invokeAgent API session ID: 7acc5df4-00b5-11f0-be7b-62f6d0df43a1
---- Step 1 ----
Took 4.3s, using 8891 tokens (in: 8799, out: 92) to complete prior action, observe, orchestrate.
To get your current energy consumption, I will call the get_consumption_statistics function and provide your customer ID.
Using tool: get_consumption_statistics with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'day': '2025/03/01', 'kind': 'measured', 'sumPowerReading': '106'}]...

---- Step 2 ----
Took 4.8s, using 9055 tokens (in: 8987, out: 68) to complete prior action, observe, orchestrate.
The function returned your energy consumption data for March 2025, which is the most recent month available in the data. To summarize it clearly:
Final response:
Based on the data provided, your current energy consumption for the month of March 2025 was 106 units....
Agent made a total of 2 L

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [41]:
forecast_agent_alias_id, forecast_agent_alias_arn = agents.create_agent_alias(
    forecast_agent[0], 'v1'
)

## Saving information
Let's store some environment variables to be used on our next notebooks.

In [42]:
forecast_agent_arn = agents.get_agent_arn_by_name(forecast_agent_name)
forecast_agent_id = forecast_agent[0]
forecast_kb = knowledge_base_name
forecast_dynamodb = dynamodb_table

%store forecast_agent_arn
%store forecast_agent_alias_arn
%store forecast_agent_alias_id
%store forecast_lambda_name
%store forecast_agent_name
%store forecast_agent_id
%store forecast_kb
%store forecast_dynamodb

Stored 'forecast_agent_arn' (str)
Stored 'forecast_agent_alias_arn' (str)
Stored 'forecast_agent_alias_id' (str)
Stored 'forecast_lambda_name' (str)
Stored 'forecast_agent_name' (str)
Stored 'forecast_agent_id' (str)
Stored 'forecast_kb' (str)
Stored 'forecast_dynamodb' (str)


In [43]:
forecast_agent_arn, forecast_agent_alias_arn, forecast_agent_alias_id

('arn:aws:bedrock:us-west-2:817724494920:agent/DIUTRVOHRF',
 'arn:aws:bedrock:us-west-2:817724494920:agent-alias/DIUTRVOHRF/CO5YH8N6BF',
 'CO5YH8N6BF')

## Next Steps
Congratulations! We've now created a forecasting agent. Next we will create our solar panel agent